In [ ]:
!pip install nltk -q
!pip install gensim -q
!pip install wordcloud -q
!pip install emoji -q
!python -m spacy download pt_core_news_sm -q
!pip install pyLDAvis -q

In [ ]:
# IMPORTAÇÃO DAS BIBLIOTECAS
from nltk.corpus import stopwords
import seaborn as sns
import pandas as pd 
import numpy as np 
import string
import re
import nltk
import emoji
import spacy 
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim import corpora, models
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
nltk.download('punkt')
nltk.download('stopwords')
pd.options.mode.chained_assignment = None  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# LEITURA DO DATAFRAME COM PANDAS
data = pd.read_csv('/content/drive/MyDrive/PesquisaCovid/db_lemmatizado/db_2020-02-01_2020-03-31__lematizado.csv', 
                   engine='python')

In [ ]:
data = data.loc[0:1000]
data

In [ ]:
# FUNÇÕES PARA CONTAGEM DE TWEETS DE ACORDO COM USUÁRIO

# Função para filtrar a conta do usuário 
def filtra_users(text):
  aux = 0
  posicao = 0
  text_temp = str(text)
  text_temp = text_temp.split()
  for word in text_temp:
    aux = aux + 1
    if (word == "'username':"):
      posicao = aux
  text_temp = text_temp[posicao]
  text_temp = re.sub(r"[',]", ' ', text_temp)
  return text_temp

# Função para agrupar todos os usuários em um único arquivo de texto
all_users = []
def agrupa_users(text):
  all_users.append(text)

# Função para contar todos os usuários em um único arquivo de texto
def conta_tweets_user(text): 
  text_temp = text.strip()
  str_all_users = ''.join(all_users)
  return str_all_users.count(text_temp)

In [ ]:
# REALIZANDO A CONTAGEM DE TWEETS POR USUÁRIO
# ATENÇÃO: A CELULA DEVE SER EXECUTADA UMA ÚNICA VEZ POR ACESSO

# Renomeando a coluna content_lemmatizada em full_text
data = data.rename(columns={'content_lemmatizada': 'full_text'})

# Criação de um novo dataframe com os dados do usuário
df = data[['full_text','user']]

# Aplicando a função para filtrar o nome de usuário dos dados
df['conta'] = df['user'].apply(filtra_users)

# Entrada de dados para agrupamento de todos os usuários
df['conta'].apply(agrupa_users)

# Função para contagem dos usúarios no agrupamento
df['qnt_tweets'] = df['conta'].apply(conta_tweets_user)

df

In [ ]:
# TRATAMENTO E VISUALIZAÇÃO DA QUANTIDADE DE TWEETS POR USUÁRIO

# Eliminando os dados duplicados 
new_df = df.drop_duplicates(subset=['conta'])

# Colocando os dados em ordem decrescente de acordom com a qtd de tweets
result_new_df = new_df[['conta','qnt_tweets']].sort_values(by=['qnt_tweets'], ascending=False)
result_new_df.reset_index(drop = True, inplace=True)

# Imprmindo os 10 primeiros resultados
#print(result_new_df.head(10))
#print(' ')

# Contando quantos usuários diferentes existem no dataframe
print('Existem '+str(result_new_df.conta.count())
      +' usuários no dataframe')

# Contando quantos usuários tem 1 tweet no dataframe
filtro_1 = result_new_df['qnt_tweets'] < 2
new_result_1 = result_new_df[filtro_1]
print(str(new_result_1.conta.count()) + ' usuários tem 1 tweet')

# Contando quantos usuários tem mais de 5 tweets no dataframe
filtro_5 = result_new_df['qnt_tweets'] >= 5 
new_result_5 = result_new_df[filtro_5]
print(str(new_result_5.conta.count()) + ' usuários tem 5 ou mais tweets')

# Contando quantos usuários tem mais de 10 tweets no dataframe
filtro_10 = result_new_df['qnt_tweets'] >= 10
new_result_10 = result_new_df[filtro_10]
print(str(new_result_10.conta.count()) + ' usuários tem 10 ou mais tweets')

result_new_df

In [ ]:
# IPRIMINDO O BOXPLOT COM A QUANTIDADE DE TWEETS POR USUÁRIO
sns.boxplot(x=result_new_df["qnt_tweets"])

In [ ]:
# FUNÇÕES PARA FILTRAR OS TWEETS DE ACORDO COM O USUÁRIO DE NOTÍCIA

# Função para selecionar e classificar os tweets como notícias
def select_contas_news(text, lista):
  inicio = []
  fim = []
  text_temp = str(text)
  text_temp = re.sub(r'["-,.:@?!&$]', ' ', text_temp)
  text_temp = text_temp.lower()
  text_temp = text_temp.split()
  inicio.append(text_temp)
  result = [word for word in text_temp if word.lower() not in lista]
  fim.append(result)
  if(np.array_equal(inicio,fim)==True):
    return text
  else:
    return 'codigo_tweet_news'

In [ ]:
# FILTRANDO OS TWEETS DE ACORDO COM O USUÁRIO
# FILTRAGEM MANUAL REALIZADA PELO DIANSLEY

# Adicionando os usuários que terão os tweets apagados
users_news = []

# Passando a lista de usuários para minúscula
new_users_news = []
for word in users_news:
  word = word.lower()
  new_users_news.append(word)

# Aplicação da função para filtrar os contas de notícias
new_data = data[['full_text','user']]
new_data['user'] = new_data['user'].apply(select_contas_news, lista=new_users_news)

# Filtra os tweets que não são de noticías
filtro = new_data.user!='codigo_tweet_news'
result_data_news = new_data[filtro]

# Reset do indice do dataframe 
result_data_news.reset_index(drop = True, inplace=True)
result_data_news['user'] = result_data_news['user'].apply(filtra_users)

result_data_news

In [ ]:
# FUNÇÕES PARA FILTRAR OS TWEETS DE ACORDO COM OS USUÁRIOS BOTS

# Função para selecionar e classificar os tweets como notícias
def select_contas_bots(text, lista):
  inicio = []
  fim = []
  text_temp = str(text)
  text_temp = re.sub(r'["-,.:@?!&$]', ' ', text_temp)
  text_temp = text_temp.lower()
  text_temp = text_temp.split()
  inicio.append(text_temp)
  result = [word for word in text_temp if word.lower() not in lista]
  fim.append(result)
  if(np.array_equal(inicio,fim)==True):
    return text
  else:
    return 'codigo_tweet_bots'

def compara_listas(lista1, lista2):
    usuarios_noticias = []
    contador = 0
    for word1 in lista1:
      if word1 in lista2:
        contador = contador + 1
    print('Existe '+str(contador)+' correspondências entres as listas'+ 
    ' de usuários de nóticia e usuários bots')


In [ ]:
# FILTRANDO OS TWEETS DE ACORDO COM O USUÁRIO BOTS
# FILTRAGEM MANUAL REALIZADA PELO GEAN ATRAVÉS DO BOTOMETER

# Adicionando os usuários que terão os tweets apagados
users_bots = []               

# Passando a lista de usuários para minúscula
new_users_bots = []
for word in users_bots:
  word = word.lower()
  new_users_bots.append(word)

# Comparando as correspondências entre as listas de notícias com bots
compara_listas(lista1=users_news, lista2=users_bots)

# Aplicação da função para filtrar os contas de notícias
result_data_news['user'] = result_data_news['user'].apply(select_contas_bots, lista=new_users_bots)

# Filtra os tweets que não são de noticías
filtro = result_data_news.user!='codigo_tweet_bots'
result_data_bots = result_data_news[filtro]

# Reset do indice do dataframe 
result_data_bots.reset_index(drop = True, inplace=True)

result_data_bots

In [ ]:
# BOXPLOT DATAFRAME APÓS REALIZR A ELIMINAÇÃO DOS USUÁRIOS DE NOTÍCIAS E BOTS

# Função para contagem dos usúarios no agrupamento
df_boxplot = result_data_bots
df_boxplot['qnt_tweets'] = df_boxplot['user'].apply(conta_tweets_user)

# Eliminando os dados duplicados 
df_boxplot = df_boxplot.drop_duplicates(subset=['user'])

# Colocando os dados em ordem decrescente de acordom com a qtd de tweets
df_boxplot = df_boxplot[['user','qnt_tweets']].sort_values(by=['qnt_tweets'], ascending=False)
df_boxplot.reset_index(drop = True, inplace=True)

# IMPRIMINDO A TABELA COM OS USUÁRIOS APÓS A LIMPEZA
df_boxplot

In [ ]:
# IPRIMINDO O BOXPLOT APÓS A LIMPEZA DE USUÁRIOS
sns.boxplot(x=df_boxplot["qnt_tweets"])

In [ ]:
# FUNÇÃO PARA CONVERSÃO DE EMOJIS EM STRINGS

# Função para converção de emojis em strings
def convert_emojis(text):
  text_temp = emoji.demojize(str(text), language='pt')
  return text_temp

In [ ]:
# CONVERTENDO OS EMOJIS EM STRINGS

# Aplicando a função de conversão de strings
result_data_bots = result_data_bots[['full_text', 'user']]
result_data_bots['full_text'] = result_data_bots['full_text'].apply(convert_emojis)

# Tratamento das string geradas na conversão dos emojis
result_data_bots['full_text'] = result_data_bots['full_text'].str.replace(':',' ')

result_data_bots

In [ ]:
# FUNÇÃO PARA LIMPEZA DOS TWEETS 
def cleaning_text(text):
  punctuation_stand = string.punctuation # Pontuações
  punctuation_others = re.compile('<.*?>') # Pontos
  urls = re.compile(r'@[a-z0-9_]+|http\S+|www\.\S+') # Usuários e url's
  text_temp = text.lower()
  text_temp = punctuation_others.sub(r'', text_temp)
  text_temp = text_temp.translate(str.maketrans('', '', string.digits)) #removendo numeros
  text_temp = urls.sub(r'', text_temp)
  text_temp = text_temp.translate(str.maketrans('','',punctuation_stand))
  return text_temp

In [ ]:
# LIMPEZA DOS TWEETS

result_data_bots['full_text'] = result_data_bots['full_text'].apply(cleaning_text)

result_data_bots

In [ ]:
# FUNÇÃO PARA SUBSTITUIÇÃO DAS PALAVRAS
def converte_palavras(palavra):
  if palavra and palavra == 'n':
    return 'nao'
  elif palavra and palavra == 'hj':
    return 'hoje'
  elif palavra and ('kk' in palavra or 'rsrs' in palavra or 'rs' == palavra or 'haha' in palavra or 'hehe' in palavra):
    return 'risos'
  elif palavra and palavra == 'nd':
    return 'nada'
  elif palavra and palavra == 'ngm':
    return 'ninguem'
  elif palavra and palavra == 'pq':
    return 'porque'
  elif palavra and palavra == 'obg':
    return 'obrigado'
  elif palavra and palavra == 'sextou':
    return 'sexta'
  elif palavra and palavra == 'sdd':
    return 'saudade'
  elif palavra and palavra == 'mt':
    return 'muito'
  elif palavra and palavra == 'tmj':
    return 'juntos'
  elif palavra and palavra == 'pfvr':
    return 'favor'
  elif palavra and palavra == 'bb':
    return 'bebe'
  elif palavra and palavra == 'víru':
    return 'virus'
  elif palavra and (palavra == 'tá' or palavra == 'ta'):
    return 'esta'
  elif palavra and (palavra == 'tô' or palavra == 'to'):
    return 'estou'
  elif palavra and (palavra == 'udi' or palavra == 'udia'):
    return 'uberlandia'
  else:
    return palavra

# FUNÇÃO PARA CHAMAR A FUNÇÃO DE SUBSTITUIÇAO DAS PALAVRAS
def substitui_palavras(text):
  text_temp = text.split()
  result = [converte_palavras(word) for word in text_temp]
  return ' '.join(result)

# CONVERSÃO DAS PALAVRAS
result_data_bots['full_text'] = result_data_bots['full_text'].apply(substitui_palavras)
result_data_bots

In [ ]:
# Conta palavras antes do processo de eliminação de palavras 0,5% e 90%
texto = []
def criaTexto(text):
  text_temp = str(text)
  texto.append(text_temp)
  return texto

nuvem_palavras_1 = result_data_bots['full_text'].apply(criaTexto)
new_texto = str(texto).strip('[]')
len(new_texto)

In [ ]:
def contemPalavraTweet(new_tweet, palavra):
  if palavra in new_tweet:
      return palavra
  else:
      return 'codigo'

# Função para selecionar e classificar os tweets como notícias
def contaQuantidade(palavra, new_tweet):
  df = new_tweet.apply(contemPalavraTweet, palavra=palavra)
  filtro = df!='codigo'
  new_df = df[filtro]
  contador = new_df.count()
  return contador
  

def selecionaDados(tweet):
  tweet_return = []
  temp = str(tweet)
  temp = temp.split()
  new_tweet = result_data_bots['full_text']
  len_df = result_data_bots['full_text'].count()
  for palavra in temp:
    cont = contaQuantidade(palavra, new_tweet)
    if (cont < len_df*0.9):
      if (cont > len_df*0.005):
        tweet_return.append(palavra)
  return ' '.join(tweet_return)

result_data_bots['full_text'] = result_data_bots['full_text'].apply(selecionaDados)

In [ ]:
# Conta palavras após o processo de eliminação de palavras 0,5% e 90%
texto = []
def criaTexto(text):
  text_temp = str(text)
  texto.append(text_temp)
  return texto

nuvem_palavras_1 = result_data_bots['full_text'].apply(criaTexto)
new_texto = str(texto).strip('[]')
new_texto

In [ ]:
# FUNÇÃO PARA FILTRAR OS TWEETS DE ACORDO COM DETERMINADA ENTRADA 

# Função para selecionar e classificar os tweets
def select_tweet_input(text, lista):
  inicio = []
  fim = []
  text_temp = str(text)
  text_temp = re.sub(r'["-,.:@?!&$]', ' ', text_temp)
  text_temp = text_temp.lower()
  text_temp = text_temp.split()
  inicio.append(text_temp)
  result = [word for word in text_temp if word.lower() not in lista]
  fim.append(result)
  if(np.array_equal(inicio,fim)==True):
    return ' '.join(result)
  else:
    return 'codigo_tweet_selected'

In [ ]:
# FILTRANDO OS TWEETS DE ACORDO COM PALAVRAS NO TEXTO
# Ex: Tweets que contenha a palavra BBB ou algum participante

# Adicionando as palavras que serão filtradas
words_input = []

# Tranformando o result_data_bots em result_clean 
result_clean = result_data_bots

# Aplicando a função para seleção dos 
result_clean['full_text'] = result_clean['full_text'].apply(select_tweet_input, lista=words_input)

# Filtra os tweets que não foram selecionados
filtro = result_clean.full_text!='codigo_tweet_selected'
result_remove = result_clean[filtro]

# Reset do indice do dataframe 
result_remove.reset_index(drop = True, inplace=True)

result_remove

In [ ]:
# FUNÇÃO PARA REMOVER DETERMINADAS PALAVRAS DOS TWEETS
def remove_words(text,lista):
  text_temp = text.split()
  result = [word for word in text_temp if word.lower() not in lista]
  result_temp = ' '.join(result)
  return result_temp

In [ ]:
# REMOVENDO AS PALAVRAS DOS TWEETS
# Ex: As palavras que foram utilizadas na coleta

# Adicionando as palavras que serão removidas
words_remove  = [ 'covid', 'covid19', 'coronavírus', 'coronavirus','pandemia',
                 'brasil','covid19brasil','sobre','ainda','pra','vai','agora','aqui','hoje','todo',
                  'dia','co','q','de','o','e','da','mil','um','a','que','uma',
                 'dos', 'é','ela','na','em','tão','do','vc','todo','toda','só',
                 'nada','tudo','no','pouco','por','nem','se','nos','para','antes',
                 'faz','neste','quer','teve','depois','mas','há','mais','ano','diz',
                 'covidbrasil','pessoa','ter','sobre','ainda','pra','vai','agora','aqui','hoje','todo',
                  'dia','https','co','CO','c o','de','mil','pessoa',
                  'tá', 'ir', 'fazer', 'poder', 'gente','falar', 'aí', 'dar',
                  'bem', 'outro', 'algum', 'não', 'vir', "…", "•", "“", "”"]

# Aplicando a função para remoção das palavras no tweet
result_remove['full_text'] = result_remove['full_text'].apply(remove_words, lista=words_remove)

result_remove

In [ ]:
def remove_stopwords(text):
  text_temp = str(text)
  stopwords = nltk.corpus.stopwords.words('portuguese') 
  newStopWords = []
  stopwords.extend(newStopWords)
  new_text = [word for word in text_temp.split() if word not in stopwords]
  return ' '.join(new_text)

In [ ]:
# Calculando a frequência de cada palavra no dataframe com NLTK
new_data_remove = result_remove['full_text']
new_data_remove = new_data_remove.apply(remove_stopwords)

sent = ' '.join(new_data_remove)
frequencia = FreqDist(word.lower() for word in word_tokenize(sent))
df_frequencia = pd.DataFrame({"Palavras": frequencia.keys(),
                              "Frequencia": frequencia.values()})
df_frequencia.nlargest(columns = "Frequencia", n = 5)

In [ ]:
# FUNÇÕES PARA TOKENIZAR, ELIMINAR STOPWORDS E LEMANTIZAR

# Função para lemantizar as palavras
def lemmatization(text):
  nlp = spacy.load('pt')
  text_temp = str(text)
  doc = nlp(text_temp)
  words_lemma = [token.lemma_ for token in doc]
  return ' '.join(words_lemma)

# Função para tokenizar e eliminar stopwords
def tokenized_words(text):
  text_temp = str(text)
  stopwords = nltk.corpus.stopwords.words('portuguese') # Stopwords
  stopwords.extend(words_remove)
  tokens = [word for word in text_temp.split() if word not in stopwords]
  return tokens

In [ ]:
# TOKENIZAÇÃO E LEMATIZAÇÃO DOS TWEETS

# Lematização dos tweets
result_remove['full_text'] = result_remove['full_text'].apply(lemmatization)

# Tokenização dos tweets
tokens = result_remove['full_text'].apply(tokenized_words)

print(tokens)

In [ ]:
# Construção do dicionário e do corpo do modelo
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(text) for text in tokens]

In [ ]:
# Treinamento do modelo LDA
ldamodel = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, passes=8)

In [ ]:
# Visualização interativa dos tópicos 
vis_data = gensimvis.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis_data)

In [ ]:
# Print dos tópicos 
ldamodel = ldamodel.show_topics(num_words=4)

for topic in ldamodel:
    print(topic)